# Faculty Scheduler

An attempt to create a faculty scheduler, which is a much easier version of the resident scheduler.

I may try to add a database instead of excel which could be editable but simpler. I could also do CSV, but I hate Excel's complaints and difficulties with saving as CSV each time.

## variables

$W_{ftd} \subset \forall \ faculty \ \times \ \forall \ teams \ \times \ \forall \ days $

$\forall \ faculty \ \exists\ \frac{s\ shifts}{yr} := F_{FTE}*s_{FTE}$

$\forall \ faculty \ \exists\ \frac{h\ holidays}{yr}\ :=\ 1$

holidays on certain days; for 11/2025 it is 11/27.

## constraints

Exactly 1 faculty per day on each team: $\sum {faculty} \ \forall \ days \forall\ teams = 1$

## objective function
Maximize value of faculty preferences.

### micro-version - month of november with Thanksgiving

In [1]:
import itertools as it

import numpy as np
import pandas as pd
from opre_tools import negated_bounded_span, print_full
from ortools.sat.python import cp_model

from datetime import date

/home/accrist/anaconda3/envs/or-tools/lib/python3.13/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
faculty = pd.read_csv("faculty_scheduling_input/faculty.csv", index_col="name")
days = pd.read_csv("faculty_scheduling_input/days.csv", index_col="date",parse_dates=["date"])
teams = pd.read_csv('faculty_scheduling_input/teams.csv', index_col="name")

,inpatient shifts per year,ideal Purple,ideal HS,clinic_only,guess_FTE
name,,,,,
Allen,0,0.000000,0.000000,clinic only,1.00
Benson,21,7.000000,14.000000,works inpatient,1.00
Eddy,14,4.666667,9.333333,works inpatient,0.50
Kerkering,0,0.000000,0.000000,clinic only,1.00
Lungren,21,7.000000,14.000000,works inpatient,1.00
May,21,7.000000,14.000000,works inpatient,1.00
McDermott,6,2.000000,4.000000,works inpatient,0.25
Potyk,6,2.000000,4.000000,works inpatient,0.25
Stewart,21,7.000000,14.000000,works inpatient,1.00


prototype accessor:
```python
var_grid.loc[pd.IndexSlice["Crist",'2025-11-01',"Orange"]]
```

let's model a toy with intervals that just has attendings who can work 7 days and 28 days total for Orange and Green

In [18]:
days_toy = pd.DataFrame({"date":pd.date_range('2025-11-01',freq="1d",periods=28), "day_idx":range(1,28+1)})
days_toy.head()

,date,day_idx
0,2025-11-01,1
1,2025-11-02,2
2,2025-11-03,3
3,2025-11-04,4
4,2025-11-05,5


In [14]:
faculty_toy = ['Crist','May','Castagna','Benson','Potyk']

In [15]:
teams = ["Orange","Green"]

In [57]:
# Parameters
attendings = ["Crist", "May", "Castagna", "Benson", "Potyk"]
days = list(range(28))
num_attendings = len(attendings)

# Decision variables: Interval variables for work periods
intervals = []
presences = {}
for a in range(num_attendings):
    num_shifts = model.NewIntVar(0, 4, f"num_shifts_{a}")  # Allow multiple work periods
    shifts = []
    for s in range(4):  # Allow up to 4 work periods per attending
        start = model.NewIntVar(0, 27, f"start_{a}_{s}")
        duration = model.NewIntVar(7, 14, f"duration_{a}_{s}")
        end = model.NewIntVar(7, 28, f"end_{a}_{s}")
        interval = model.NewIntervalVar(start, duration, end, f"interval_{a}_{s}")
        shifts.append(interval)

        for d in days:
            presences[(a, s, d)] = model.NewBoolVar(f"presence_{a}_{s}_{d}")
            model.Add(start <= d).OnlyEnforceIf(presences[(a, s, d)])
            model.Add(d < end).OnlyEnforceIf(presences[(a, s, d)])
            model.Add(start > d).OnlyEnforceIf(presences[(a, s, d)].Not())
            model.Add(d >= end).OnlyEnforceIf(presences[(a, s, d)].Not())

    intervals.extend(shifts)
    model.Add(num_shifts == sum(1 for s in range(4))).OnlyEnforceIf(
        model.NewBoolVar(f"has_shift_{a}_{s}")
    )

# Constraint: Each day must be covered by exactly one attending
for d in days:
    model.Add(
        sum(presences[(a, s, d)] for a in range(num_attendings) for s in range(4)) == 1
    )

# Constraint: Ensure non-overlapping work periods for the same attending
for a in range(num_attendings):
    model.AddNoOverlap(intervals[a * 4 : (a + 1) * 4])

model.SetName("Faculty Scheduler")
print(model.ModelStats())

satisfaction model 'Faculty Scheduler': (model_fingerprint: 0x595c1d93deb1bdfc)
#Variables: 911
  - 818 Booleans in [0,1]
  - 6 in [0,4]
  - 29 in [0,27]
  - 29 in [7,14]
  - 29 in [7,28]
#kInterval: 29
#kLinear1: 3'253 (#enforced: 3'253)
#kLinearN: 56 (#terms: 700)
#kNoOverlap: 6 (#intervals: 25, #variable_sizes: 25)


In [58]:
# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Print the schedule
if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    for d in days:
        working_attendings = [attendings[a] for a in range(num_attendings) for s in range(4) if solver.Value(presences[(a, s, d)])]
        print(f"Day {d+1}: {', '.join(working_attendings)}")
else:
    print("No feasible solution found.")

No feasible solution found.


In [53]:
print(solver.ResponseStats())

CpSolverResponse summary:
status: INFEASIBLE
objective: NA
best_bound: NA
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.0118011
usertime: 0.0118028
deterministic_time: 0
gap_integral: 0

